## Presentation Type Classifier (with Hidden Types)

#### Also Network Visualization with Louvain Communities--See below

* **Updated June 2022**
* **Key Features**:

    * Uses `getDistance` to identify `close matches` with side-by-side comparison of soggetti.  
    * For example: With a distance of "1", the soggetti `4, 1, 2, 3`, and `5, 1, 2, 3` will count as the same. These are reported as "flexed entries" in a separate column.
    * Finds time intervals between entries (expressed as offsets, like `8.0, 4.0, 8.0`)
    * Finds melodic intervals between first note of successive entries in each pattern (like `P-5, P-8`)
    * Labels Fuga, PEn, and ID according to time intervals between entries. NIm not yet supported!
    * If two entries are separated by more than 10 bars (80 offsets), the tool resets to a new pattern
    * Counts number of entries
    * Provides offset and measure/beat locations
    * Sorts all presentation types by the order in which they appear in the piece
    * Reports voice names of the entries, in order of their appearance
    * Omits singleton soggetti (just one entry of a given motive in isolation)

In [1]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy as np
import itertools
from itertools import combinations
import networkx as nx
from community import community_louvain
from copy import deepcopy
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


## A.  Start by Finding Presentation Types in a Corpus of Pieces 

- Results are combined into a single dataframe
- Option to save combined results as CSV

In [2]:
prefix = 'https://crimproject.org/mei/'

piece_list = ['CRIM_Model_0019',
             'CRIM_Mass_0019_1',
             'CRIM_Mass_0019_2',
             'CRIM_Mass_0019_3',
             'CRIM_Mass_0019_4',
             'CRIM_Mass_0019_5']

suffix = '.mei'

In [4]:

final = pd.DataFrame()

for piece in piece_list:
    file_url = prefix + piece 
    url = prefix + piece + suffix
    piece = importScore(url)   
    output = piece.presentationTypes(limit_to_entries = True,
                                      edit_distance_threshold = 0,
                                    include_hidden_types = False,
                                    combine_unisons = True,
                                   melodic_ngram_length = 4)

    final['Validation'] = ""
    final['Comments'] = ""
    final = final.append(output, ignore_index=True)
final.head()
# final.to_csv("corpus_classified.csv")
# final.to_csv("saved_csv/CRIM_Mass_0019_P_Types_hidden.csv")

Successfully imported https://crimproject.org/mei/CRIM_Model_0019.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0019_1.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0019_2.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0019_3.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0019_4.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0019_5.mei


,Validation,Comments,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries
0,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0, 11/3.0, 14/1.0, 1...","[P-5, P-4, P-5, P8, P5, P-8]","[0.0, 8.0, 40.0, 48.0, 84.0, 104.0, 128.0]","[(-3, 3, 2, -2)]","[8.0, 32.0, 8.0, 36.0, 20.0, 24.0]","[Cantus, Altus, Tenor, Bassus, Altus, Cantus, ...",FUGA,7.0,False
1,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,150.0,"[19/4.0, 21/4.0, 22/4.0, 25/4.0, 27/4.0, 29/4....","[P8, P-5, P-4, P-5, P8, P8, P-11]","[150.0, 166.0, 174.0, 198.0, 214.0, 230.0, 250...","[(-3, 2, 2, -2)]","[16.0, 8.0, 24.0, 16.0, 16.0, 20.0, 24.0]","[Tenor, Cantus, Altus, Tenor, Bassus, Altus, C...",FUGA,8.0,False
2,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,298.0,"[38/2.0, 38/4.0, 42/2.0, 44/1.0, 46/2.0, 48/4.0]","[P-5, P-5, P8, P8, P-11]","[298.0, 302.0, 330.0, 344.0, 362.0, 382.0]","[(-3, 3, -2, -2)]","[4.0, 28.0, 14.0, 18.0, 20.0]","[Altus, Tenor, Bassus, Altus, Cantus, Bassus]",FUGA,6.0,False
3,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,394.0,"[50/2.0, 51/4.0, 53/2.0, 55/2.0, 56/4.0, 59/1....","[P5, P-8, P4, P5, P-8, P8, P-4, P-5]","[394.0, 406.0, 418.0, 434.0, 446.0, 464.0, 486...","[(-2, 2, 3, -2)]","[12.0, 12.0, 16.0, 12.0, 18.0, 22.0, 4.0, 16.0]","[Altus, Cantus, Bassus, Tenor, Altus, Bassus, ...",FUGA,9.0,False
4,,,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0]","[P-5, P-4, P-5]","[0.0, 8.0, 40.0, 48.0]","[(-3, 3, 2, -2)]","[8.0, 32.0, 8.0]","[Cantus, Altus, Tenor, Bassus]",ID,4.0,False


## B. Network Visualization with Louvain Communities

* This will only make sense for a corpus!

## Add Combined Data for Melodic, Time and Soggetti as "Words" for Network Edges

* We combine the lists of time and melodic intervals of entries as strings (thus a kind of word that uniquely identifies that pattern)
* We do the same for soggetti, combining the melodic intervals themselves as a string
* These are in turn assembled in various combinations:  TINT, MINT, and SOG, also as strings

In [7]:
final["MINT"] = final["Melodic_Entry_Intervals"].apply(joiner)
final["TINT"] = final["Time_Entry_Intervals"].apply(joiner)

final['SOG'] = final['Soggetti'].apply(clean_melody_new)
final['ALL'] = final["MINT"] + '_' + final["TINT"] + '_' + final['SOG']

final["ALL_INT"] = final["MINT"] + '_' + final["TINT"]
final["MINT_SOG"] = final["MINT"] + '_' + final["SOG"]
final["TINT_SOG"] = final["TINT"] + '_' + final["SOG"]
final.head()

,Validation,Comments,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,...,Presentation_Type,Number_Entries,Flexed_Entries,MINT,TINT,SOG,ALL,ALL_INT,MINT_SOG,TINT_SOG
0,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0, 11/3.0, 14/1.0, 1...","[P-5, P-4, P-5, P8, P5, P-8]","[0.0, 8.0, 40.0, 48.0, 84.0, 104.0, 128.0]","[(-3, 3, 2, -2)]","[8.0, 32.0, 8.0, 36.0, 20.0, 24.0]",...,FUGA,7.0,False,P-5_P-4_P-5_P8_P5_P-8,8.0_32.0_8.0_36.0_20.0_24.0,-3_3_2_-2,P-5_P-4_P-5_P8_P5_P-8_8.0_32.0_8.0_36.0_20.0_2...,P-5_P-4_P-5_P8_P5_P-8_8.0_32.0_8.0_36.0_20.0_24.0,P-5_P-4_P-5_P8_P5_P-8_-3_3_2_-2,8.0_32.0_8.0_36.0_20.0_24.0_-3_3_2_-2
1,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,150.0,"[19/4.0, 21/4.0, 22/4.0, 25/4.0, 27/4.0, 29/4....","[P8, P-5, P-4, P-5, P8, P8, P-11]","[150.0, 166.0, 174.0, 198.0, 214.0, 230.0, 250...","[(-3, 2, 2, -2)]","[16.0, 8.0, 24.0, 16.0, 16.0, 20.0, 24.0]",...,FUGA,8.0,False,P8_P-5_P-4_P-5_P8_P8_P-11,16.0_8.0_24.0_16.0_16.0_20.0_24.0,-3_2_2_-2,P8_P-5_P-4_P-5_P8_P8_P-11_16.0_8.0_24.0_16.0_1...,P8_P-5_P-4_P-5_P8_P8_P-11_16.0_8.0_24.0_16.0_1...,P8_P-5_P-4_P-5_P8_P8_P-11_-3_2_2_-2,16.0_8.0_24.0_16.0_16.0_20.0_24.0_-3_2_2_-2
2,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,298.0,"[38/2.0, 38/4.0, 42/2.0, 44/1.0, 46/2.0, 48/4.0]","[P-5, P-5, P8, P8, P-11]","[298.0, 302.0, 330.0, 344.0, 362.0, 382.0]","[(-3, 3, -2, -2)]","[4.0, 28.0, 14.0, 18.0, 20.0]",...,FUGA,6.0,False,P-5_P-5_P8_P8_P-11,4.0_28.0_14.0_18.0_20.0,-3_3_-2_-2,P-5_P-5_P8_P8_P-11_4.0_28.0_14.0_18.0_20.0_-3_...,P-5_P-5_P8_P8_P-11_4.0_28.0_14.0_18.0_20.0,P-5_P-5_P8_P8_P-11_-3_3_-2_-2,4.0_28.0_14.0_18.0_20.0_-3_3_-2_-2
3,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,394.0,"[50/2.0, 51/4.0, 53/2.0, 55/2.0, 56/4.0, 59/1....","[P5, P-8, P4, P5, P-8, P8, P-4, P-5]","[394.0, 406.0, 418.0, 434.0, 446.0, 464.0, 486...","[(-2, 2, 3, -2)]","[12.0, 12.0, 16.0, 12.0, 18.0, 22.0, 4.0, 16.0]",...,FUGA,9.0,False,P5_P-8_P4_P5_P-8_P8_P-4_P-5,12.0_12.0_16.0_12.0_18.0_22.0_4.0_16.0,-2_2_3_-2,P5_P-8_P4_P5_P-8_P8_P-4_P-5_12.0_12.0_16.0_12....,P5_P-8_P4_P5_P-8_P8_P-4_P-5_12.0_12.0_16.0_12....,P5_P-8_P4_P5_P-8_P8_P-4_P-5_-2_2_3_-2,12.0_12.0_16.0_12.0_18.0_22.0_4.0_16.0_-2_2_3_-2
4,,,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0]","[P-5, P-4, P-5]","[0.0, 8.0, 40.0, 48.0]","[(-3, 3, 2, -2)]","[8.0, 32.0, 8.0]",...,ID,4.0,False,P-5_P-4_P-5,8.0_32.0_8.0,-3_3_2_-2,P-5_P-4_P-5_8.0_32.0_8.0_-3_3_2_-2,P-5_P-4_P-5_8.0_32.0_8.0,P-5_P-4_P-5_-3_3_2_-2,8.0_32.0_8.0_-3_3_2_-2


### Make Groups, Based on Selected Criteria:

* MINT
* TINT
* SOG
* Or any of the combinations:
    * ALL (MINT + TINT + Soggetti)
    * ALL_INT (MINT + TINT)
    * MINT_SOG (MINT + Soggetti)
    * TINT_SOG (TINT + Soggetti)

In [8]:

output_grouped = final.groupby('SOG')['Title'].apply(list).reset_index()
output_grouped


,SOG,Title
0,-2_-2_-2_-2,"[Missa Veni sponsa Christi: Credo, Missa Veni ..."
1,-2_-2_-2_2,[Missa Veni sponsa Christi: Credo]
2,-2_-2_3_2,[Missa Veni sponsa Christi: Credo]
3,-2_-3_3_-2,[Missa Veni sponsa Christi: Gloria]
4,-2_2_-2_-2,[Missa Veni sponsa Christi: Credo]
5,-2_2_-3_2,"[Missa Veni sponsa Christi: Credo, Missa Veni ..."
6,-2_2_2_-2,[Missa Veni sponsa Christi: Agnus Dei]
7,-2_2_2_2,"[Missa Veni sponsa Christi: Kyrie, Missa Veni ..."
8,-2_2_3_-2,"[Veni sponsa Christi, Missa Veni sponsa Christ..."
9,-3_2_2_-2,"[Veni sponsa Christi, Missa Veni sponsa Christ..."


### Group and Find Unique Pairs

In [9]:
pairs = output_grouped.Title.apply(lambda x: list(combinations(x, 2)))
pairs2 = pairs.explode().dropna()
unique_pairs = pairs.explode().dropna().unique()

In [10]:
pd.Series(unique_pairs).isna().sum()

0

In [11]:
def add_communities(G):
    G = deepcopy(G)
    partition = community_louvain.best_partition(G)
    nx.set_node_attributes(G, partition, "group")
    return G
pyvis_graph = Network(notebook=True, width="1800", height="1400", bgcolor="white", font_color="black")
G = nx.Graph()
G.add_edges_from(unique_pairs)
G = add_communities(G)
pyvis_graph.from_nx(G)
pyvis_graph.show('MINT.html')